# GPT2 Using GAAM as a Text Classifier

## Install Required Libraries


In [10]:
!pip install datasets # unified interface for accessing and working with various datasets (by hugging face)
!pip install -U accelerate # library to optimize and accelerate numerical computations
!pip install -U transformers # library by hugging face that gives easy access to pre-trained models, tokenizers, and tools for fine-tuning models

import utils # some utility functions we wrote that are used across the different notebooks

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Loading and Processing the Dataset

We load the dataset from Hugging Face. Each sample consists of one strings feature that stores the title as well as the (start of the) article-text. The label is the category that the article belongs to (world, sports, business, sci/tech). [Link](https://huggingface.co/datasets/ag_news/viewer/default/train) to explore the structure of the data.

In [11]:
from datasets import load_dataset

dataset = load_dataset('ag_news')

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


Reduce the size of the dataset (to reduce training times) whilst ensuring that the original structure and distribution of the data is kept.

In [12]:
from datasets import Dataset, DatasetDict
import pandas as pd

# from utils import take_a_percentage_of_data

dataset_train_1percent = utils.take_a_percentage_of_data(dataset['train'], percentage=0.001)
dataset_test_1percent = utils.take_a_percentage_of_data(dataset['test'], percentage=0.001)

dataset_1percent = DatasetDict({
    'train': dataset_train_1percent,
    'test': dataset_test_1percent
}) # combine the shortened datasets back into the old structure.

print(dataset_1percent)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
})


## Tokenizing the dataset

Tokenize the dataset in the exact same way as the GPT-2 model.

In [13]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # padding tokens added to sequences will be represented by an end-of-sequence token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length")

tokenized_dataset = dataset_1percent.map(tokenize_function, batched=True) # performed in batches to increase performance

print(tokenized_dataset) # tokenization adds two features: 'input_ids' (the tokenized representation of 'text') as well as 'attention_mask', which ensures that the model does not attend to padding tokens added during tokenization

Map: 100%|██████████| 4/4 [00:00<00:00, 297.05 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 120
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4
    })
})


In [14]:
print(len(tokenized_dataset["test"]["attention_mask"][1]))

1024


## Loading the Pre-trained GPT2-Model

Load the pre-trained GPT2-Model for sequence classification.

In [15]:
from transformers import GPT2ForSequenceClassification

gpt2_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=4) # our gpt2-model should distinguish between 4 labels, adds a final fully connected layers with 4 output neurons.

print(gpt2_model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


## Implementing the Gaussian Adaptive Attention Block

Download the package provided by the paper.

In [16]:
!pip3 install gaussian-adaptive-attention

Defaulting to user installation because normal site-packages is not writeable


**Approach 1**: Import the GaussianBlock directly from the package and replace the current attention mechanism in the GPT2-model with it.

In [17]:
import importlib
gaussian_adaptive_attention = importlib.import_module("gaussian_adaptive_attention")
GaussianBlock = getattr(gaussian_adaptive_attention, "GaussianBlock")
import torch
import copy

gaussian_model = copy.deepcopy(gpt2_model)

num_layers = 1

for block in gaussian_model.transformer.h: # accessing each transformer blocks within the GPT-2 model
  block.attn = GaussianBlock(
    num_layers=num_layers,  # Number of layers in the GPT-2 model
    norm_axes=[1] * num_layers,  # Assuming layer normalization along the sequence length dimension
    num_heads=[5] * num_layers,  # Number of attention heads
    num_gaussians=[4] * num_layers  # Number of Gaussian components
  )

print(gaussian_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GaussianBlock(
          (layers): ModuleList(
            (0): MultiHeadGaussianAdaptiveAttention(
              (attention_heads): ModuleList(
                (0-4): 5 x GaussianAdaptiveAttention()
              )
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


However, this model doesn't work, because keyword-arguments are passed to the attention mechanism that the GaussianBlock cannot handle (e.g. the argument `last_layer`), which is why the GaussianBlock returns an error.

**Approach 2**: Create a wrapper class of MultiHeadGaussianAdaptive Attention that is then inserted into the GPT2-architecture layer by layer.

The `forward`-method of the wrapper class accepts additional keyword arguments, so the error is not thrown. However, the values of the keyword arguments are currently not actively used in the function, since we saw no meaningful way to integrate them into the Gaussian attention mechanism.

In [18]:
import importlib
gaussian_adaptive_attention = importlib.import_module("gaussian_adaptive_attention")
MultiHeadGaussianAdaptiveAttention = getattr(gaussian_adaptive_attention, "MultiHeadGaussianAdaptiveAttention")
import torch
import copy

gaussian_model = copy.deepcopy(gpt2_model)

class MultiHeadGaussianAdaptiveAttentionWrapper(torch.nn.Module):
    def __init__(self, config, num_heads=4, num_gaussians=5, norm_axis=1):
        super().__init__()
        self.attention = MultiHeadGaussianAdaptiveAttention(
            norm_axis=norm_axis,
            num_heads=num_heads,
            num_gaussians=num_gaussians,
            padding_value=config.eos_token_id,
            eps=config.layer_norm_epsilon
        )

    def forward(self, hidden_states, **kwargs):
        # Pass arguments using **kwargs to the underlying attention mechanism
        attention_output = self.attention(hidden_states)
        return (hidden_states,) + tuple(attention_output)  # Ensure the return value is a tuple


# Replace the attention mechanism in each transformer block
for block in gaussian_model.transformer.h: # accessing each transformer blocks within the GPT-2 model
    block.attn = MultiHeadGaussianAdaptiveAttentionWrapper(config=gaussian_model.config, num_heads=4, num_gaussians=5, norm_axis=1) # and replacing the attention module with the Gaussian attention block.


## Hyperparameter Tuning

The hyperparameters of the GAAM-block  are

In [27]:
possible_norm_axis = [0, 1, 2]
possible_num_gaussians = [5, 10, 20, 40]
possible_num_heads = [4, 8, 16, 32]

models_with_different_parameters = {}

for norm_axis in possible_norm_axis:
  for comb in range(len(possible_num_gaussians)):
    var_name = f"gaussian_gpt2_model_axis_{norm_axis}_gaussians_{possible_num_gaussians[comb]}_heads_{possible_num_heads[comb]}"
    gaussian_model = copy.deepcopy(gpt2_model)
    for block in gaussian_model.transformer.h: # accessing each transformer blocks within the GPT-2 model
      block.attn = MultiHeadGaussianAdaptiveAttentionWrapper(config=gaussian_model.config, num_heads=possible_num_heads[comb], num_gaussians=possible_num_gaussians[comb], norm_axis=norm_axis) # and replacing the attention module with the Gaussian attention block.
    models_with_different_parameters[var_name] = gaussian_model
    if comb == 0:
      test_model = gaussian_model

print(models_with_different_parameters)

{'gaussian_gpt2_model_axis_0_gaussians_5_heads_4': GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadGaussianAdaptiveAttentionWrapper(
          (attention): MultiHeadGaussianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-3): 4 x GaussianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
), 'gaussian_gpt2

: 

In [20]:
# from transformers import Trainer, TrainingArguments

# accuracy = torch.zeros(12)
# validation_loss = torch.zeros(12)

# for model_name, network in models_with_different_parameters.items():

#     training_args = TrainingArguments(
#         output_dir="./results",
#         learning_rate=0.00002,
#         per_device_train_batch_size=1,
#         per_device_eval_batch_size=1,
#         num_train_epochs=1,
#         use_cpu = False,
#         no_cuda = False,
#         save_strategy= 'epoch',
#         logging_strategy = 'epoch',
#         evaluation_strategy='epoch',
#         load_best_model_at_end = True
#         # save_steps = 100,
#         #logging_steps = 100,
#     )

#     trainer = Trainer(
#         model=network,
#         args=training_args,
#         train_dataset=tokenized_dataset["train"],
#         eval_dataset=tokenized_dataset["test"],
#         compute_metrics=utils.compute_accuracy
#     )

#     trainer.train()

    

In [21]:
print(gaussian_model)
print(gpt2_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadGaussianAdaptiveAttentionWrapper(
          (attention): MultiHeadGaussianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-31): 32 x GaussianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (

## Freezing Parameters

For the first training epochs, we decided to freeze the pre-trained layers. We did this, because we want the attention weights to first learn some general things before we fine-tune the model as a whole.

In [22]:
gaussian_model = utils.freeze_pretrained_layers(test_model)

In [23]:
trainable_params = sum(p.numel() for p in gaussian_model.parameters() if p.requires_grad)
frozen_params = sum(p.numel() for p in gaussian_model.parameters() if not p.requires_grad)

print(f"Trainable Parameters: {trainable_params}")
print(f"Frozen Parameters: {frozen_params}")

Trainable Parameters: 33792
Frozen Parameters: 96091392


## Check: Do both models run?

1. The Normal GPT2-Model
2. The GPT2-Model with GAAM

In [24]:
input_ids = torch.randint(0, gpt2_model.config.vocab_size, (1, 512))
labels = torch.tensor([1]).unsqueeze(0)

outputs = gpt2_model(input_ids=input_ids, labels=labels)
loss, logits = outputs['loss'], outputs['logits']
print(f"GPT2 with regular attention mechanism: Loss = {round(loss.item(), 4)}, logits = {logits.detach()}")

outputs = gaussian_model(input_ids=input_ids, labels=labels)
loss, logits = outputs['loss'], outputs['logits']
print(f"GPT2 with Gaussian attention mechanism: Loss = {round(loss.item(), 4)}, logits = {logits.detach()}")


GPT2 with regular attention mechanism: Loss = 0.0008, logits = tensor([[-2.5849,  5.8351, -4.4555, -1.6336]])
GPT2 with Gaussian attention mechanism: Loss = 0.9784, logits = tensor([[ 0.1357,  0.9590,  0.8093, -0.0615]])


## Training

Training the GPT2-Model with Gaussian attention

In [25]:
from transformers import Trainer, TrainingArguments

def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=0.00002,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    use_cpu = False,
    no_cuda = False,
    save_strategy= 'epoch',
    logging_strategy = 'epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end = True
    # save_steps = 100,
    #logging_steps = 100,
)

trainer = Trainer(
    model=gaussian_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_accuracy
)


2024-05-13 20:25:59.083847: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
model_name = "gpt2-ag_news-1percent-frozen"

trainer.train()
trainer.save_model(model_name)
# Save the tokenizer used by the model as well
tokenizer.save_pretrained(model_name)


  0%|          | 0/120 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
best_accuracy = trainer.state.best_metric
print("Best accuracy:", best_accuracy)

print(trainer.state.eval_accuracy)

## Calculating the accuracy

In [ ]:
import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gaussian_model.to(device)

data = dataset_1percent['test']

# Calculate accuracy
accuracy = utils.calculate_accuracy(gaussian_model, data, tokenizer)
print(f"Accuracy: {accuracy:.4f}")

## Second Training Cycle with Unfrozen Layers

Loading the previously trained model and unfreezing the layers.

In [ ]:

unfrozen_gaussian_model = copy.deepcopy(gaussian_model)
print(unfrozen_gaussian_model)

# unfreeze layers
for param in unfrozen_gaussian_model.transformer.parameters():
    param.requires_grad = True

# check
trainable_params = sum(p.numel() for p in unfrozen_gaussian_model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadGaussianAdaptiveAttentionWrapper(
          (attention): MultiHeadGaussianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x GaussianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)
Trainable Parameters: 96095904


Defining the new parameters for the second training cycle (smaller learning rate, including weight decay)

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    use_cpu = False,
    no_cuda = False,
)

trainer = Trainer(
    model=unfrozen_gaussian_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


NameError: name 'unfrozen_gaussian_model' is not defined

Starting the second training cycle.

In [ ]:
trainer.train()

Step,Training Loss
500,1.405000
1000,1.399500
1500,1.395700
2000,1.391200
2500,1.396400
3000,1.389600
3500,1.390000
4000,1.390000
4500,1.393400
5000,1.387900


TrainOutput(global_step=6000, training_loss=1.3929286905924478, metrics={'train_runtime': 2274.1474, 'train_samples_per_second': 2.638, 'train_steps_per_second': 2.638, 'total_flos': 2090634706944000.0, 'train_loss': 1.3929286905924478, 'epoch': 5.0})

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unfrozen_gaussian_model.to(device)

# Test the function with an example from the dataset
text = dataset['test'][1]['text']
label = dataset['test'][1]['label']
print("Predicted label:", utils.predict_label(text))
print("Expected label:", label)


Predicted label: 2
Expected label: 3


In [ ]:
from torch.utils.data import DataLoader

data = dataset_1percent['test']

# Calculate accuracy
accuracy = utils.calculate_accuracy(unfrozen_gaussian_model, data, tokenizer)
print(f"Accuracy: {accuracy:.4f}")

NameError: name 'unfrozen_gaussian_model' is not defined

## Alternative: Training a GPT2-Model from Scratch

In [ ]:
from transformers import GPT2ForSequenceClassification, GPT2Config

# Define GPT-2 configuration
config = GPT2Config(
    vocab_size=50257,  # Number of tokens in the vocabulary
    n_embd=768,        # Dimensionality of the embeddings and hidden states
    n_layer=12,        # Number of transformer layers
    n_head=12,         # Number of attention heads
    num_labels=4       # Number of labels for sequence classification
)

gaussian_gpt2_model_untrained = GPT2ForSequenceClassification(config)

print(gaussian_gpt2_model_untrained)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


Replace the attention layer with a Gaussian attention layer.

In [ ]:
# Replace the attention mechanism in each transformer block
for block in gaussian_gpt2_model_untrained.transformer.h: # accessing each transformer blocks within the GPT-2 model
    block.attn = MultiHeadGaussianAdaptiveAttentionWrapper(config=gaussian_model.config, num_gaussians=5, norm_axis=1) # and replacing the attention module with the Gaussian attention block.

print(gaussian_gpt2_model_untrained)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadGaussianAdaptiveAttentionWrapper(
          (attention): MultiHeadGaussianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x GaussianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


Train the Gaussian model from scratch.

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    # weight_decay=0.01,
    use_cpu = False,
    no_cuda = False
)

trainer = Trainer(
    model=gaussian_gpt2_model_untrained,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

Step,Training Loss
500,1.865900
1000,1.517300
1500,1.450000
2000,1.437400
2500,1.421200
3000,1.403500
3500,1.414400
4000,1.400900
4500,1.404300
5000,1.395500


TrainOutput(global_step=6000, training_loss=1.4576284484863282, metrics={'train_runtime': 2184.9509, 'train_samples_per_second': 2.746, 'train_steps_per_second': 2.746, 'total_flos': 2090634706944000.0, 'train_loss': 1.4576284484863282, 'epoch': 5.0})